# RAPTOR: 재귀적 추상적 처리 및 주제별 조직화(RAPTOR) 기반 검색 시스템

## 개요
RAPTOR는 계층적 문서 요약, 임베딩 기반 검색, 문맥적 응답 생성 기능을 결합한 고급 정보 검색 및 질문-응답 시스템입니다. 대규모 문서 집합을 효율적으로 처리하기 위해 다층 트리 구조의 요약본을 생성하여, 넓은 범위와 세부적인 정보 검색을 모두 가능하게 합니다.

## 동기
기존 검색 시스템은 대규모 문서 집합을 다루는 데 있어 중요한 세부 정보를 놓치거나 불필요한 정보로 인해 과부하가 발생하는 문제를 자주 겪습니다. RAPTOR는 문서 집합의 계층적 구조를 만들어 상위 개념과 특정 세부 정보를 상황에 맞게 탐색할 수 있도록 하여 이러한 문제를 해결합니다.

## 주요 구성 요소
1. **트리 빌딩**: 문서 요약의 계층적 구조 생성
2. **임베딩 및 클러스터링**: 의미적 유사성을 기반으로 문서와 요약 정리
3. **벡터 저장소**: 문서와 요약의 임베딩을 효율적으로 저장하고 검색
4. **문맥적 검색기**: 특정 쿼리에 맞는 가장 관련성 높은 정보 선택
5. **응답 생성**: 검색된 정보를 바탕으로 일관된 답변 생성

## 방법 상세 설명

### 트리 빌딩
1. 레벨 0에서는 원본 문서로 시작합니다.
2. 각 레벨에서 다음 과정을 수행합니다:
   - 언어 모델을 사용하여 텍스트 임베딩 생성
   - 임베딩 클러스터링(예: Gaussian Mixture Models 사용)
   - 각 클러스터에 대해 요약본 생성
   - 생성된 요약본을 다음 레벨의 텍스트로 사용
3. 하나의 요약본에 도달하거나 최대 레벨에 도달할 때까지 반복

### 임베딩 및 검색
1. 트리의 모든 레벨의 문서와 요약본을 임베딩합니다.
2. 이러한 임베딩을 벡터 저장소(예: FAISS)에 저장하여 유사성 검색을 빠르게 수행합니다.
3. 쿼리가 주어지면:
   - 쿼리를 임베딩합니다.
   - 벡터 저장소에서 가장 유사한 문서/요약을 검색합니다.

### 문맥적 압축
1. 검색된 문서/요약본을 가져옵니다.
2. 언어 모델을 사용하여 쿼리에 가장 관련성이 높은 부분만 추출합니다.

### 응답 생성
1. 관련된 부분을 결합하여 문맥을 구성합니다.
2. 이 문맥과 원래의 쿼리를 바탕으로 언어 모델을 사용해 답변을 생성합니다.

## 접근 방식의 장점
1. **확장성**: 다양한 수준의 요약을 사용하여 대규모 문서 집합을 처리할 수 있습니다.
2. **유연성**: 높은 수준의 개요와 특정 세부 정보를 모두 제공할 수 있습니다.
3. **문맥 인식**: 추상화 수준에 따라 가장 적합한 정보를 검색합니다.
4. **효율성**: 임베딩과 벡터 저장소를 활용하여 빠른 검색을 수행합니다.
5. **추적 가능성**: 요약본과 원본 문서 간의 링크를 유지하여 출처 확인이 가능합니다.

## 결론
RAPTOR는 정보 검색 및 질문-응답 시스템에 있어서 중요한 진전을 나타냅니다. 계층적 요약과 임베딩 기반 검색 및 문맥적 응답 생성을 결합하여, 대규모 문서 집합을 효과적으로 다룰 수 있는 강력하고 유연한 접근 방식을 제공합니다. 이 시스템의 계층적 추상화 탐색 기능을 통해 폭넓은 쿼리에 대해 관련성 높고 문맥적으로 적합한 답변을 제공할 수 있습니다.

RAPTOR는 앞으로 트리 빌딩 프로세스 최적화, 요약 품질 향상, 복잡하고 다면적인 쿼리를 더 잘 처리할 수 있는 검색 메커니즘 개선 등에 대한 추가 연구가 필요합니다. 또한, 다른 AI 기술과 결합함으로써 더욱 정교한 정보 처리 시스템을 개발할 수 있는 가능성도 열려 있습니다.


<div style="text-align: center;">

<img src="../images/raptor.svg" alt="RAPTOR" style="width:100%; height:auto;">
</div>

### Imports and Setup


In [1]:
import numpy as np
import pandas as pd
from typing import List, Dict, Any
from sklearn.mixture import GaussianMixture
from langchain.chains.llm import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema import AIMessage
from langchain.docstore.document import Document

import matplotlib.pyplot as plt
import logging
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_3933/1959881810.py:22: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *
/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 1.4.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Define logging, llm and embeddings

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4o-mini")

### Helper Functions


In [3]:
def extract_text(item):
    """Extract text content from either a string or an AIMessage object."""
    # `item`이 AIMessage 객체인지 확인하고, 그렇다면 `content` 속성을 반환합니다.
    # 그렇지 않으면 `item` 자체를 반환하여 일반 문자열도 처리할 수 있게 합니다.
    if isinstance(item, AIMessage):
        return item.content
    return item

# aimessage 텍스트에 대해 임베딩을 실시 
def embed_texts(texts: List[str]) -> List[List[float]]:
    logging.info(f"Embedding {len(texts)} texts")  # 로그 기록: 임베딩할 텍스트 수 표시
    return embeddings.embed_documents([extract_text(text) for text in texts])

# 주어진 임베딩 배열들에 대해 클러스터링함  -> 주어진 임베딩이 어느 클러스터에 속해있는지 레이블 반환 
def perform_clustering(embeddings: np.ndarray, n_clusters: int = 10) -> np.ndarray:
    logging.info(f"Performing clustering with {n_clusters} clusters")  # 클러스터 수 표시
    gm = GaussianMixture(n_components=n_clusters, random_state=42)  # GMM 객체 생성
    return gm.fit_predict(embeddings)  # 클러스터링 수행 후, 각 데이터 포인트의 클러스터 레이블 반환

# 입력된 텍스트 리스트를 요약함 
def summarize_texts(texts: List[str]) -> str:
    """Summarize a list of texts using OpenAI."""
    logging.info(f"Summarizing {len(texts)} texts")  # 요약할 텍스트 수 표시
    prompt = ChatPromptTemplate.from_template(
        "Summarize the following text concisely:\n\n{text}"
    )
    chain = prompt | llm  # 프롬프트와 LLM을 연결하여 체인 생성
    input_data = {"text": texts}  # 입력 데이터 생성
    return chain.invoke(input_data)  # 체인 호출하여 요약 결과 반환

# pca로 2차원으로 임베딩을 축소하여 시각화함 
def visualize_clusters(embeddings: np.ndarray, labels: np.ndarray, level: int):
    from sklearn.decomposition import PCA  # PCA 클래스 임포트
    pca = PCA(n_components=2)  # 2개의 주성분으로 차원 축소 설정
    reduced_embeddings = pca.fit_transform(embeddings)  # 임베딩 차원 축소
    
    plt.figure(figsize=(10, 8))  # 플롯 크기 설정
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='viridis')
    plt.colorbar(scatter)  # 컬러 바 추가하여 각 클러스터 색상 구분
    plt.title(f'Cluster Visualization - Level {level}')  # 제목에 클러스터 레벨 표시
    plt.xlabel('First Principal Component')  # x축 레이블
    plt.ylabel('Second Principal Component')  # y축 레이블
    plt.show()  # 플롯 출력


### RAPTOR Core Function


In [4]:
def build_raptor_tree(texts: List[str], max_levels: int = 3) -> Dict[int, pd.DataFrame]:
    # 각 레벨의 메타데이터와 부모-자식 관계를 포함하는 RAPTOR 트리 구조를 생성합니다.
    results = {}  # 각 레벨별 데이터를 저장할 딕셔너리
    current_texts = [extract_text(text) for text in texts]  # 초기 텍스트 리스트 생성
    current_metadata = [{"level": 0, "origin": "original", "parent_id": None} for _ in texts]
    # 각 텍스트에 대해 초기 메타데이터 설정 (레벨 0, 원본 데이터, 부모 없음)

# 레벨 1부터 시작 
    for level in range(1, max_levels + 1):
        logging.info(f"Processing level {level}")  # 현재 레벨 

        # 텍스트 임베딩 생성
        embeddings = embed_texts(current_texts)
        
        # 클러스터 수 설정 (최소 10개, 또는 현재 텍스트 수의 절반 이하)
        n_clusters = min(10, len(current_texts) // 2)
        
        # Gaussian Mixture Model을 사용하여 클러스터링 수행
        cluster_labels = perform_clustering(np.array(embeddings), n_clusters)

        # 현재 레벨의 데이터프레임 생성 및 결과 저장
        df = pd.DataFrame({
            'text': current_texts,
            'embedding': embeddings,
            'cluster': cluster_labels,
            'metadata': current_metadata
        })
        results[level-1] = df  # 각 레벨의 결과 저장

        summaries = []  # 요약본을 저장할 리스트
        new_metadata = []  # 새 메타데이터 저장할 리스트

        # 현재 레벨의 각 클러스터별 텍스트 추출, 메타데이터 추출  -> 각 클러스의 텍스트를 요약 후 리스트에 넣음 
        for cluster in df['cluster'].unique():
            # 각 클러스터에 대해 텍스트와 메타데이터를 추출하고 요약을 생성
            cluster_docs = df[df['cluster'] == cluster]
            cluster_texts = cluster_docs['text'].tolist()
            cluster_metadata = cluster_docs['metadata'].tolist()
            
            # 클러스터 텍스트 요약 생성-> 각 레벨 별 모든 클러스터에 대한 텍스트의 요약을 리스트로 한다. -> 즉, 요약 하나에 메타데이터 하나 
            summary = summarize_texts(cluster_texts)
            summaries.append(summary)
            
            # 요약본에 대한 메타데이터 생성
            new_metadata.append({
                "level": level,
                "origin": f"summary_of_cluster_{cluster}_level_{level-1}",
                "child_ids": [meta.get('id') for meta in cluster_metadata], # 자식레벨에서 사용했던 id -> 클러스터링 된 것들 
                "id": f"summary_{level}_{cluster}" # 현 부모 id-> 클러스터 대표 id 
            })

        # 다음 레벨에 사용할 텍스트와 메타데이터 업데이트 -> 각 클러스터별 요약된 내용, 메타데이터가 새롭게 정의된다. 
        current_texts = summaries
        current_metadata = new_metadata # 다음 메타데이터 

        # 요약본이 하나만 남았을 경우 트리 생성 중단
        if len(current_texts) <= 1:
            results[level] = pd.DataFrame({
                'text': current_texts,
                'embedding': embed_texts(current_texts),
                'cluster': [0],
                'metadata': current_metadata
            })
            logging.info(f"Stopping at level {level} as we have only one summary")  # 중단 기록
            break

    return results  # 각 레벨의 데이터프레임을 포함한 최종 트리 구조 반환 -> 레벨 별 텍스트의 요약과 메타데이터와 그에 해당하는 임베딩 및 클러스터가 들어있음 


### Vectorstore Function


In [5]:
def build_vectorstore(tree_results: Dict[int, pd.DataFrame]) -> FAISS:
    """Build a FAISS vectorstore from all texts in the RAPTOR tree."""
    all_texts = [] #
    all_embeddings = []
    all_metadatas = []
    
    for level, df in tree_results.items():
        all_texts.extend([str(text) for text in df['text'].tolist()])  # 모든 텍스트를 저장 
        all_embeddings.extend([embedding.tolist() if isinstance(embedding, np.ndarray) else embedding for embedding in df['embedding'].tolist()]) # 모든 임베딩을 저장 
        all_metadatas.extend(df['metadata'].tolist()) # 모든 메타데이터를 저장 
    
    logging.info(f"Building vectorstore with {len(all_texts)} texts")
    
    # 모든 문서와 메타데이터에 있는 내용과 메타데이터를 뽑아서 리스트화 
    documents = [Document(page_content=str(text), metadata=metadata) 
                 for text, metadata in zip(all_texts, all_metadatas)]
    
    # 문서와 임베딩에 대해 FAISS에 저장 
    return FAISS.from_documents(documents, embeddings)

### Define tree traversal retrieval

In [6]:
# 최상위 레벨에서 시작하여 하위 라벨로 이동하면서 세부 정보를 검색 
# 각 레벨에서 자식 ID(child_ids)를 추출하여 쿼리와 연관된 문서들만 하위 레벨로 넘깁니다.
# 쿼리와 유사한 상위 요약본과 함께, 하위 레벨의 더 세부적인 정보를 포함한 문서들을 모두 반환합니다.


def tree_traversal_retrieval(query: str, vectorstore: FAISS, k: int = 3) -> List[Document]:
    # 입력된 쿼리에 대한 임베딩 생성
    query_embedding = embeddings.embed_query(query)
    
    # 특정 레벨에서 문서 검색
    def retrieve_level(level: int, parent_ids: List[str] = None) -> List[Document]:
        # 부모 ID가 주어졌을 경우, 해당 부모 ID와 레벨에 맞는 문서 검색
        # 부모 레벨에서 유사한 문서들을 검색함 
        if parent_ids:
            docs = vectorstore.similarity_search_by_vector_with_relevance_scores(
                query_embedding,
                k=k,
                filter=lambda meta: meta['level'] == level and meta['id'] in parent_ids
            )
        else:
            # 부모 ID가 없는 경우, 현재 레벨에 해당하는 모든 문서 중 유사도 기반으로 검색
            docs = vectorstore.similarity_search_by_vector_with_relevance_scores(
                query_embedding,
                k=k,
                filter=lambda meta: meta['level'] == level
            )
        
        # 현재 레벨에서 검색 결과가 없거나 레벨이 0이면 현재 문서를 반환
        if not docs or level == 0:
            return docs
        
        # 검색된 문서들에서 자식 ID 수집
        child_ids = [doc.metadata.get('child_ids', []) for doc, _ in docs]
        child_ids = [item for sublist in child_ids for item in sublist]  # 리스트 평탄화
        
        # 하위 레벨에서 자식 ID를 기반으로 재귀적으로 문서 검색
        child_docs = retrieve_level(level - 1, child_ids)
        return docs + child_docs  # 현재 레벨 문서와 하위 레벨 문서를 합쳐 반환
    
    # 트리의 최상위 레벨을 얻기 위해 벡터 저장소의 모든 문서 메타데이터에서 최대 레벨 찾기
    max_level = max(doc.metadata['level'] for doc in vectorstore.docstore.values())
    return retrieve_level(max_level)  # 최상위 레벨에서 검색 시작


### Create Retriever


In [7]:
def create_retriever(vectorstore: FAISS) -> ContextualCompressionRetriever:
    logging.info("Creating contextual compression retriever")
    base_retriever = vectorstore.as_retriever()
    
    # 문서와 질문이 주어져 있을 때 질문에 답변하기 위해 관련된 정보만 뽑기 
    prompt = ChatPromptTemplate.from_template(
        "Given the following context and question, extract only the relevant information for answering the question:\n\n"
        "Context: {context}\n"
        "Question: {question}\n\n"
        "Relevant Information:"
    )
    
    extractor = LLMChainExtractor.from_llm(llm, prompt=prompt)
    
    # 전체 문서에서 관련 문서를 찾고 찾은 문서에서 관련된 부분만 추출함
    return ContextualCompressionRetriever(
        base_compressor=extractor,
        base_retriever=base_retriever
    )


### Define hierarchical retrieval

In [8]:
def hierarchical_retrieval(query: str, retriever: ContextualCompressionRetriever, max_level: int) -> List[Document]:
    """Perform hierarchical retrieval starting from the highest level, handling potential None values."""
    all_retrieved_docs = []
    
    for level in range(max_level, -1, -1):
        # 현재 레벨에서 쿼리에 맞는 문서만 추출함 
        level_docs = retriever.get_relevant_documents(
            query,
            filter=lambda meta: meta['level'] == level
        )
        all_retrieved_docs.extend(level_docs)
        
        # 하위 레벨 검색을 위해 자식 id 수집 
        if level_docs and level > 0:
            child_ids = [doc.metadata.get('child_ids', []) for doc in level_docs]
            child_ids = [item for sublist in child_ids for item in sublist if item is not None]  # Flatten and filter None
            
            # 자식 id가 있을 때 쿼리에 자식 쿼리를 합쳐서 전달함 
            if child_ids: 
                child_query = f" AND id:({' OR '.join(str(id) for id in child_ids)})"
                query += child_query
    
    return all_retrieved_docs # 최종 합친 문서 출력 

### RAPTOR Query Process (Online Process)

In [14]:
def raptor_query(query: str, retriever: ContextualCompressionRetriever, max_level: int) -> Dict[str, Any]:
    """Process a query using the RAPTOR system with hierarchical retrieval."""
    logging.info(f"Processing query: {query}")  # 쿼리 처리 시작 로그 기록
    
    # 1. 계층적 검색 수행
    relevant_docs = hierarchical_retrieval(query, retriever, max_level)  # 주어진 쿼리에 맞는 문서 검색
    
    # 2. 검색된 문서의 세부 정보 저장
    doc_details = []
    for i, doc in enumerate(relevant_docs, 1):
        doc_details.append({
            "index": i,  # 문서 인덱스
            "content": doc.page_content,  # 문서의 내용
            "metadata": doc.metadata,  # 메타데이터 (레벨, 출처 등)
            "level": doc.metadata.get('level', 'Unknown'),  # 문서의 레벨 정보 (알 수 없는 경우 'Unknown')
            "similarity_score": doc.metadata.get('score', 'N/A')  # 유사도 점수 (없으면 'N/A')
        })
    
    # 3. 답변 생성에 사용할 컨텍스트 설정
    context = "\n\n".join([doc.page_content for doc in relevant_docs])  # 각 문서의 내용을 컨텍스트로 합침
    
    # 4. 답변 생성을 위한 프롬프트 설정
    prompt = ChatPromptTemplate.from_template(
        "Given the following context, please answer the question:\n\n"
        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer:"
    )
    
    # 프롬프트에 따라 LLM과의 체인 설정
    chain = LLMChain(llm=llm, prompt=prompt)
    answer = chain.run(context=context, question=query)  # 설정한 체인으로 답변 생성
    
    logging.info("Query processing completed")  # 쿼리 처리 완료 로그 기록
    
    # 5. 결과 반환
    result = {
        "query": query,  # 쿼리 내용
        "retrieved_documents": doc_details,  # 검색된 문서들의 상세 정보
        "num_docs_retrieved": len(relevant_docs),  # 검색된 문서의 개수
        "context_used": context,  # 답변 생성에 사용된 전체 컨텍스트
        "answer": answer,  # 생성된 답변
        "model_used": llm.model_name,  # 사용된 모델 이름
    }
    
    return result  # 최종 결과 반환


def print_query_details(result: Dict[str, Any]):
    """Print detailed information about the query process, including tree level metadata."""
    print(f"Query: {result['query']}")  # 쿼리 출력
    print(f"\nNumber of documents retrieved: {result['num_docs_retrieved']}")  # 검색된 문서 개수 출력
    print(f"\nRetrieved Documents:")
    
    # 각 검색된 문서의 세부 정보 출력
    for doc in result['retrieved_documents']:
        print(f"  Document {doc['index']}:")
        print(f"    Content: {doc['content'][:100]}...")  # 문서 내용의 첫 100자 출력
        print(f"    Similarity Score: {doc['similarity_score']}")  # 유사도 점수 출력
        print(f"    Tree Level: {doc['metadata'].get('level', 'Unknown')}")  # 문서 레벨 정보 출력
        print(f"    Origin: {doc['metadata'].get('origin', 'Unknown')}")  # 문서의 출처 정보 출력
        if 'child_docs' in doc['metadata']:
            print(f"    Number of Child Documents: {len(doc['metadata']['child_docs'])}")  # 자식 문서 수 출력
        print()  # 빈 줄 추가로 문서 구분
    
    # 답변 생성에 사용된 전체 컨텍스트 출력
    print(f"\nContext used for answer generation:")
    print(result['context_used'])
    
    # 생성된 답변 출력
    print(f"\nGenerated Answer:")
    print(result['answer'])
    
    # 사용된 모델 이름 출력
    print(f"\nModel Used: {result['model_used']}")


## Example Usage and Visualization


## Define data folder

In [11]:
path = "../data/Understanding_Climate_Change.pdf"

### Process texts

In [12]:
loader = PyPDFLoader(path)
documents = loader.load()
texts = [doc.page_content for doc in documents]

### Create RAPTOR components instances

In [13]:
# Build the RAPTOR tree
tree_results = build_raptor_tree(texts)

2024-11-02 03:00:48,867 - INFO - Processing level 1
2024-11-02 03:00:48,868 - INFO - Embedding 33 texts
2024-11-02 03:00:50,102 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-02 03:00:51,386 - INFO - Performing clustering with 10 clusters
2024-11-02 03:00:56,257 - INFO - Summarizing 1 texts
2024-11-02 03:00:58,434 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:00:58,456 - INFO - Summarizing 2 texts
2024-11-02 03:01:01,296 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:01:01,302 - INFO - Summarizing 3 texts
2024-11-02 03:01:04,674 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:01:04,677 - INFO - Summarizing 4 texts
2024-11-02 03:01:10,169 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:01:10,172 - INFO - Summarizing 9 texts


In [15]:
# Build vectorstore
vectorstore = build_vectorstore(tree_results)

2024-11-02 03:07:43,281 - INFO - Building vectorstore with 48 texts
2024-11-02 03:07:44,538 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-02 03:07:45,047 - INFO - Loading faiss.
2024-11-02 03:07:45,093 - INFO - Successfully loaded faiss.


In [16]:
# Create retriever
retriever = create_retriever(vectorstore)

2024-11-02 03:07:45,110 - INFO - Creating contextual compression retriever


### Run a query and observe where it got the data from + results

In [17]:
# Run the pipeline
max_level = 3  # Adjust based on your tree depth
query = "What is the greenhouse effect?"
result = raptor_query(query, retriever, max_level)
print_query_details(result)

2024-11-02 03:07:45,119 - INFO - Processing query: What is the greenhouse effect?
/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_8259/3372560863.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  level_docs = retriever.get_relevant_documents(
2024-11-02 03:07:45,869 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-02 03:07:46,721 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:07:47,713 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:07:51,049 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:07:52,321 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-02 03:07:53,148 - INFO - HTTP Request: POST h

Query: What is the greenhouse effect?

Number of documents retrieved: 16

Retrieved Documents:
  Document 1:
    Content: The greenhouse effect is caused by greenhouse gases, such as carbon dioxide, methane, and nitrous ox...
    Similarity Score: N/A
    Tree Level: 1
    Origin: summary_of_cluster_8_level_0

  Document 2:
    Content: The provided context does not contain information about the greenhouse effect. It primarily discusse...
    Similarity Score: N/A
    Tree Level: 0
    Origin: original

  Document 3:
    Content: The greenhouse effect is a natural process where greenhouse gases, such as carbon dioxide (CO2), met...
    Similarity Score: N/A
    Tree Level: 0
    Origin: original

  Document 4:
    Content: The context does not explicitly define the greenhouse effect. However, it mentions greenhouse gas em...
    Similarity Score: N/A
    Tree Level: 2
    Origin: summary_of_cluster_2_level_1

  Document 5:
    Content: The greenhouse effect refers to the process by whi